In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Clone Transformers and Prepare env

In [ ]:
%cd /content

/content/transformers


In [ ]:
!git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git


Cloning into 'transformers'...
remote: Enumerating objects: 138580, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 138580 (delta 1), reused 47 (delta 1), pack-reused 138533
Receiving objects: 100% (138580/138580), 145.05 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (104082/104082), done.


In [ ]:
%cd /content/transformers
!pip3 install --upgrade .

/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7416891 sha256=d71e5889462c10e05fbe6b981da412e69c1cd10a06905e78eb991bda9001d629
  Stored in directory: /tmp/pip-ephem-wheel-cache-bj6g1woy/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers


In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.8 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=a501479726c11bb95d3163dd57003f20b971280ce0c85686b94ecbd23660ed00
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00


#Preprocessing Data

In [ ]:
def xoa_trung_lap(s):
  loop = ""
  le = len(s)
  i=1
  while i <= len(s)-1:
    if s[i]==s[i-1] and (i==len(s)-1 or s[i+1]==' '):
      j=i
      loop=s[i]
      while s[j-1] == s[j]:
        loop+=s[j]
        j-=1
      s = s.replace(loop, s[i])
    i+=1
  return s

print(xoa_trung_lap("thầy rất dễ thươngggggggggg đó là điều mà em không bao giờ nói  dạy học mà như dân chợ búaaaaa"))

thầy rất dễ thương đó là điều mà em không bao giờ nói  dạy học mà như dân chợ búa


In [ ]:
#load dictionary guid
import pandas as pd
dic_guid = pd.read_csv('/content/drive/MyDrive/dict_guid/DicGuid.csv')
dic_guid

,word,target
0,good,tốt
1,zn,cảm
2,ez,dễ
3,zời,vời
4,full,toàn
...,...,...
101,"""yêu""",không thích
102,"""được""",bị
103,"""sớm""",muộn
104,quas,quá


In [ ]:
def map2good(sen, dic):
  for i in dic.values:
    sen=sen.replace(i[0], i[1])
  return sen

In [ ]:
sentence = 'được học nhiều good nghiệm từ thầy wzjwz163 .'
def scanerr(sentence):
  sentence=sentence.lower()
  #chuyen tu tieng viet, teencode, tu kho hieu sang dang de hon
  #sentence = map2good(sentence, dic_guid)

  #xoa ky tu nghi la spam
  tem=''
  i=0
  while sentence[i] != '.' and sentence[i] != '!' and i<len(sentence)-1:

    if sentence[i] == 'w' or sentence[i] == 'z' or sentence[i]=='f' or sentence[i]=='j':
      tem = sentence[i]
      j=i
      while sentence[j-1] != ' ':
        tem = sentence[j-1]+tem
        j-=1
      while sentence[j+1] != ' ' and j+1 < len(sentence)-1:
        tem = tem + sentence[j+1]
        j+=1
    i+=1

  sentence=sentence.replace(tem, '')
  #xoa ky tu dac biet
  sentence=sentence.replace(":3", '')
  sentence=sentence.replace("<3", '')
  sentence=sentence.replace(":>", '')
  sentence=sentence.replace(":v", '')
  sentence=sentence.replace(":)", '')
  sentence=sentence.replace("=)", '')
  sentence=sentence.replace(")", '')
  sentence=sentence.replace(":(", '')
  sentence=sentence.replace("(", '')
  sentence=sentence.replace("!", '')
  sentence=sentence.replace("?", '')
  sentence=sentence.replace(",", '')
  sentence=sentence.replace("'", '')
  sentence=sentence.replace('"', '')
  sentence=sentence.replace('^', '')
  sentence=sentence.replace('_', '')

  sentence = xoa_trung_lap(sentence)

  return sentence

In [ ]:
scanerr(sentence)

'được học nhiều tốt nghiệm từ thầy wzjwz163 .'

In [ ]:
#word_segment
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content')

rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

output = rdrsegmenter.word_segment(text)

print(output)

['Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .', 'Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .']


#TXT2CSV

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/train.txt", sep="\t", header=None)
df[:5]

,0,1
0,positive,tài liệu được bổ sung nhiều và đầy đủ trên moo...
1,positive,giảng viên phải nên cho ví dụ cụ thể những phầ...
2,positive,"bài tập đa dạng , cụ thể ."
3,positive,slide up đầy đủ trước mỗi buổi học .
4,negative,"nội dung giảng dạy quá nhàm chán , khô khan ."


In [ ]:
train_df=df[:4000].dropna(axis=0)

In [ ]:
len(train_df)

3997

In [ ]:
test_df = df[4000:].dropna(axis=0)

In [ ]:
len(test_df)

1000

In [ ]:
def process_data2(row):
  sentence = scanerr(row[1])
  sen_af = rdrsegmenter.word_segment(sentence)
  label = 0
  if row[0] == 'negative':
    label = 0
  elif row[0] == 'positive':
    label = 1
  else:
    label = 2
  return sen_af[0], label

In [ ]:
arr1 = []
arr2 = []
for i in train_df.values:
  x, y = process_data2(i)
  arr1.append(x)
  arr2.append(y)

In [ ]:
d={'text':arr1, 'label': arr2}
df_csv = pd.DataFrame(data=d)

In [ ]:
df_csv[:5]

,text,label
0,tài_liệu được bổ_sung nhiều và đầy_đủ trên moo...,1
1,giảng_viên phải nên cho ví_dụ cụ_thể những phầ...,1
2,bài_tập đa_dạng cụ_thể .,1
3,slide up đầy_đủ trước mỗi buổi học .,1
4,nội_dung giảng_dạy quá nhàm_chán khô_khan .,0


In [ ]:
arr3 = []
arr4 = []
for i in test_df.values:
  x, y = process_data2(i)
  arr3.append(x)
  arr4.append(y)

In [ ]:
dt={'text':arr3, 'label': arr4}
dft_csv = pd.DataFrame(data=dt)

In [ ]:
dft_csv[:5]

,text,label
0,ngoài_ra cũng cần có tiết học để sinh_viên đượ...,0
1,nên để sinh_viên tự tìm_hiểu lý_thuyết trước ở...,0
2,nhập_tâm .,1
3,nên cho bài_tập hàng tuần sát với nội_dung bài...,1
4,cô giải ít bài_tập .,1


In [ ]:
%cd /content/drive/MyDrive
!mkdir data_tempo
%cd /content

/content/drive/MyDrive
/content


In [ ]:
df_csv.to_csv("/content/drive/MyDrive/data_tempo/train.csv", index=False)

In [ ]:
dft_csv.to_csv("/content/drive/MyDrive/data_tempo/test.csv", index=False)

#Loading Model to Train

In [ ]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer=AutoTokenizer.from_pretrained("vinai/phobert-base")

#Load data

In [ ]:
from datasets import load_dataset
data_files = {"train": "train.csv", "test": "test.csv"}
train_dataset = load_dataset("/content/drive/MyDrive/data_temp", split="train")
val_dataset = load_dataset("/content/drive/MyDrive/data_temp", split="test")

FileNotFoundError: ignored

In [ ]:
print(val_dataset[2], train_dataset[2])

{'text': 'bài_tập đa_dạng , cụ_thể .', 'label': 1} {'text': 'bạn nào khá thì có _ thể áp _ dụng_công _ thức vào bài _ tập cả lớp chưa đến 30% còn thực _ sự hiểu môn _ học thì rất ít người hiểu', 'label': 0}


#Traning

In [ ]:
from typing import List
from transformers import TrainingArguments, Trainer

In [ ]:
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)


train_dataset = train_dataset.map(tokenization, batched = True, batch_size = len(train_dataset))
val_dataset = val_dataset.map(tokenization, batched = True, batch_size = len(val_dataset))

Map:   0%|          | 0/3502 [00:00<?, ? examples/s]

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
  output_dir='./results',
  num_train_epochs=3,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=16,
  remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.471900
1000,0.319700


TrainOutput(global_step=1314, training_loss=0.3531654160498121, metrics={'train_runtime': 280.295, 'train_samples_per_second': 37.482, 'train_steps_per_second': 4.688, 'total_flos': 631678787691588.0, 'train_loss': 0.3531654160498121, 'epoch': 3.0})

In [ ]:
device = torch.device("cuda")
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
#predict thu 1 example
import numpy as np
ans=[]
sentence = 'Trường Đại học Khoa học Tự nhiên'
be = sentence
sentence = scanerr(sentence)
sen_af = rdrsegmenter.word_segment(sentence)
input_ids = torch.tensor([tokenizer.encode(sen_af[0])]).to(device)


with torch.no_grad():

    out = model(input_ids)
    if np.argmax(out.logits.softmax(dim=-1).tolist())==0:
      ans.append('negative')
    elif np.argmax(out.logits.softmax(dim=-1).tolist())==1:
      ans.append('positive')
    else:
      ans.append('neutral')
    print(np.argmax(out.logits.softmax(dim=-1).tolist()))
    print(be)
    print(sentence)
    print(out.logits.softmax(dim=-1).tolist())
    print(ans)

2
Trường Đại học Khoa học Tự nhiên
trường đại học khoa học tự nhiên
[[0.039241716265678406, 0.08462852984666824, 0.8761298060417175]]
['neutral']


In [ ]:
model.save_pretrained('/content/out_model') #dir to the folder save model

In [ ]:
path = '/content/drive/MyDrive/Poto/model1.pth'
torch.save(model, path)

#Evaluate

In [ ]:
#evaluate model vua train
trainer.evaluate()

{'eval_loss': 0.4974914491176605,
 'eval_accuracy': 0.8878504672897196,
 'eval_f1': 0.8233965268448027,
 'eval_precision': 0.8258778258778259,
 'eval_recall': 0.8221429986135869,
 'eval_runtime': 0.3258,
 'eval_samples_per_second': 328.472,
 'eval_steps_per_second': 21.489,
 'epoch': 3.0}

In [ ]:
#load model tu file output vua luu
model1 = RobertaForSequenceClassification.from_pretrained("/content/out_model")
model1.cuda()

In [ ]:
#load model da duoc train truoc day
model2 = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/retrained")
model2.cuda()

In [ ]:
import numpy as np

In [ ]:
ans = []
ans1 = []
ans2 = []
eval = []
eval1 = []
eval2 = []

for sentence in val_dataset['text']:
  be = sentence
  sentence = scanerr(sentence)
  sen_af = rdrsegmenter.word_segment(sentence)
  input_ids = torch.tensor([tokenizer.encode(sen_af[0])]).to(device)

  with torch.no_grad():
      out = model(input_ids)
      if np.argmax(out.logits.softmax(dim=-1).tolist())==0:
        ans.append('negative')
      elif np.argmax(out.logits.softmax(dim=-1).tolist())==1:
        ans.append('positive')
      else:
        ans.append('neutral')
      eval.append(np.argmax(out.logits.softmax(dim=-1).tolist()))
      out1 = model1(input_ids)
      if np.argmax(out1.logits.softmax(dim=-1).tolist())==0:
        ans1.append('negative')
      elif np.argmax(out1.logits.softmax(dim=-1).tolist())==1:
        ans1.append('positive')
      else:
        ans1.append('neutral')
      eval1.append(np.argmax(out1.logits.softmax(dim=-1).tolist()))
      out2 = model2(input_ids)
      if np.argmax(out2.logits.softmax(dim=-1).tolist())==0:
        ans2.append('negative')
      elif np.argmax(out2.logits.softmax(dim=-1).tolist())==1:
        ans2.append('positive')
      else:
        ans2.append('neutral')
      eval2.append(np.argmax(out2.logits.softmax(dim=-1).tolist()))
      print(be)
      print(sentence)
      print(ans[-1])
      print(ans1[-1])
      print(ans2[-1])


tài_liệu được bổ_sung nhiều và đầy_đủ trên moodle .
tàiliệu được bổsung nhiều và đầyđủ trên moodle .
positive
positive
positive
giảng_viên phải nên cho ví_dụ cụ_thể những phần quan_trọng nên nói kỹ có_thể lấy ví_dụ minh_hoạ tránh việc lập đi lập lại nhiều lần sẽ tạo cảm_giác chán cho sinh_viên .
giảngviên phải nên cho vídụ cụthể những phần quantrọng nên nói kỹ cóthể lấy vídụ minhhoạ tránh việc lập đi lập lại nhiều lần sẽ tạo cảmgiác chán cho sinhviên .
negative
negative
negative
bài_tập đa_dạng , cụ_thể .
bàitập đadạng  cụthể .
negative
negative
negative
slide up đầy_đủ trước mỗi buổi học .
slide up đầyđủ trước mỗi buổi học .
positive
positive
neutral
nội_dung giảng_dạy quá nhàm_chán , khô_khan .
nộidung giảngdạy quá nhàmchán  khôkhan .
negative
negative
negative
thầy chép bảng nhiều .
thầy chép bảng nhiều .
negative
negative
neutral
thầy tận_tình hướng_dẫn cách làm , giải_quyết những thắc_mắc của sinh_viên .
thầy tậntình hướngdẫn cách làm  giảiquyết những thắcmắc của sinhviên .
positi

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(eval,val_dataset['label'], average='macro'))
print(f1_score(eval1,val_dataset['label'], average='macro'))
print(f1_score(eval2,val_dataset['label'], average='macro'))


0.6484509145203434
0.6850843584055926
0.7077414455463237


#Infer

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment") #day la default model, ban co the folder chua model khac

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/data_phase_2.txt", sep="\t", header=None)
print(test_df[0][:5])

0                     Tiến sĩ câu giờ, hoàng đế gây mê
1    Mới học 2 bữa thôi là đã nghiện cách giảng dạy...
2     kêu giơ tay phát biểu cộng điểm nhưng có cái nịt
3               thầy mãi là number one trong lòng em !
4    Thầy như là người được chọn, sỡ hữu haki bá vư...
Name: 0, dtype: object


In [ ]:
test_df = ["Tiến sĩ câu giờ, hoàng đế gây mê", "thầy này dạy chán quá"]

In [ ]:
import numpy as np
ans = []
for sentence in test_df[0]:
  be = sentence
  sentence = scanerr(sentence)
  sen_af = rdrsegmenter.word_segment(sentence)
  input_ids = torch.tensor([tokenizer.encode(sen_af[0])]).to(device)


  with torch.no_grad():
      out = model(input_ids)
      if np.argmax(out.logits.softmax(dim=-1).tolist())==0:
        ans.append('negative')
      elif np.argmax(out.logits.softmax(dim=-1).tolist())==1:
        ans.append('positive')
      else:
        ans.append('neutral')
      print(be)
      print(sentence)
      print(np.argmax(out.logits.softmax(dim=-1).tolist()))




T
t
2
i
i
2
ế
ế
0
n
n
2


IndexError: ignored

In [ ]:
with open("/content/submit.txt", "w") as txt_file:
    for line in ans:
        txt_file.write("".join(line) + "\n")

#TEsst convert from pytorch to tf

In [ ]:
pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 24.8 MB/s eta 0:00:00


In [ ]:
pip install onnx-tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 10.9 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import sys

import cv2
import numpy as np
import onnx
import torch
import tensorflow as tf
from PIL import Image
from torchvision import transforms
from torchvision.models import *
from torchsummary import summary
from onnx_tf.backend import prepare


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
def convert_torch_to_onnx(onnx_path, image_path, model=None, torch_path=None):
    """
        Coverts Pytorch model file to ONNX
        :param torch_path: Torch model path to load
        :param onnx_path: ONNX model path to save
        :param image_path: Path to test image to use in export progress
    """
    if torch_path is not None:
        pytorch_model = get_torch_model(torch_path)
    else:
        pytorch_model = model

    image, _, torch_image = get_example_input(image_path)

    torch.onnx.export(
        model = pytorch_model,
        args = torch_image,
        f = onnx_path,
        verbose = False,
        export_params=True,
        do_constant_folding = False,
        input_names = ['input'],
        opset_version = 10,
        output_names = ['output'])


In [ ]:
def get_example_input(image_file):
    """
        Load image from disk and converts to compatible shape
        :param image_file: Path to single image file
        :return: Orginal image, numpy.ndarray instance image, torch.Tensor image
    """
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ])

    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(image)
    torch_img = transform(pil_img)
    torch_img = torch_img.unsqueeze(0)
    torch_img = torch_img.to(torch.device("cpu"))
    print(torch_img.shape)
    return image, torch_img.numpy(), torch_img


In [ ]:

onnx_model_path ='/content/drive/MyDrive/Poto/model.onnx'



In [ ]:
pip install onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.6/535.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 29.6 MB/s eta 0:00:00


In [ ]:
import onnx

In [ ]:
input_text = "This."
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=64, truncation=True, pad_to_max_length=True)
input_tensor = torch.tensor([input_ids])

# Export the model to ONNX
onnx_path = "roberta_sequence_classification.onnx"
torch.onnx.export(model, input_tensor, onnx_model_path, verbose=True)

# Load the ONNX model
onnx_model = onnx.load(onnx_model_path)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
pip install tf2onnx tensorflow onnx


In [ ]:
from onnx_tf.backend import prepare

tf_rep = prepare(onnx_model)

In [ ]:
tf_model_path = "/content/drive/MyDrive/Musi"
tflite_model_path = "/content/drive/MyDrive/Musi/assets/converted_model.tflite"

In [ ]:
tf_model_path_new = "/content/drive/MyDrive/Musi"

In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open(tflite_model_path, "wb").write(tflite_model)

IsADirectoryError: ignored

#new test

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

(…)ation/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

(…)-translation/resolve/main/tokenizer.json:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

(…)ion/resolve/main/special_tokens_map.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

(…)it5-translation/resolve/main/config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

In [ ]:
# Dummy input data for the model
input_data = "This is a test sentence"
inputs = tokenizer(input_data, return_tensors="pt")


# Export the model to ONNX
onnx_path = "envit5_translation.onnx"
dynamic_axes = {"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}}
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    onnx_path,
    dynamic_axes=dynamic_axes,
    verbose=True
)

NameError: ignored